In [184]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool

In [ ]:
import requests
from langchain_core.tools import tool
# 지역명 location 으로 위도 경도 알아내는 api
# location='서울시 구로구'

@tool    # 함수 테스트하고 싶으면 tool 데코레이터 주석처리 하기
def  get_coordinates(location):
  """
    주어진 장소 이름을 기반으로 위도와 경도를 조회합니다.
    OpenStreetMap Nominatim API를 사용합니다.
  """
  url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
  headers = {
    "User-Agent": "LangChainApp/1.0 (your@email.com)"   # http 요청 헤더:서버에게 요청을 보낸 클라이언트의 정보
  }
  res = requests.get(url, headers=headers).json()  # res 는 요청에 대한 응답
  if res:
    # print(res)  # 응답 출력
    lat = res[0]['lat']
    lon = res[0]['lon']
    print(f"{location}의 위도는 {lat}, 경도는 {lon}입니다.")
    return f"{lat},{lon}"
  else:
    # print("좌표를 찾을 수 없습니다.")
    return "좌표를 찾을 수 없습니다."


In [186]:
# get_coordinates('서울시 구로구')    # @tool 은 직접 호출할 수 없음.(TypeError: 'StructuredTool' object is not callable)

In [187]:
import os
import json

@tool  # description 속성 또는 아래와 같은 docstring 작성해야 함.
def get_weather_info(lat_lon):
  """
    get_coordinates 함수에서 리턴한 위도와 경도가 있는 문자열을 전달 받아서
    api.openweathermap.org API 를 사용하여 날씨를 구합니다.
  """
  # lat_lon = '37.5666791,126.9782914'
  lat, lon = lat_lon.split(',')
  print(lat,lon)
  api_key = os.getenv('OPENWEATHER_API_KEY')
  url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=kr"
  res = requests.get(url).json()
  if 'weather' in res:
      weather_desc = res['weather'][0]['description']
      temp = res['main']['temp']
      feels_like = res['main']['feels_like']
      humidity = res['main']['humidity']
      
      result = (
        f"날씨: {weather_desc}\n" 
        f"기온(°C): {temp}°C\n"
        f"체감온도(°C): {feels_like}°C\n"
        f"습도(%): {humidity}%\n"
        f"지역: {res.get('name', 'Unknown')}"
        )
      
      # 결과는 문자열로 리턴
      return result
  else:
      # print("날씨 정보를 불러올 수 없습니다.")
      return "날씨 정보를 불러올 수 없습니다."

# get_weather_info('37.4951999,126.8877000')

In [188]:
llm = ChatOpenAI(model="gpt-4.1-mini")
tools = [get_weather_info, get_coordinates]
llm_with_tools = llm.bind_tools(tools)
tool_dict = {
    'get_weather_info': get_weather_info,
    'get_coordinates': get_coordinates
}

In [189]:
# user_input="뉴욕 날씨는 어때?"
user_input="서울 날씨 알려줘"
messages = [
    SystemMessage(content="너는 날씨를 알려주는 기상캐스터 봇이야." \
    "도구에서 받은 날씨 정보를 사람에게 말하듯 자연스럽게 요약해서 알려줘."),
    HumanMessage(content="서울 날씨 알려줘"),
]


In [190]:
response = llm_with_tools.invoke(messages)
messages.append(response)
print(type(response))
print(response.tool_calls)

<class 'langchain_core.messages.ai.AIMessage'>
[{'name': 'get_coordinates', 'args': {'location': '서울'}, 'id': 'call_8jWOpy7xZxpOeOwTovNez7PE', 'type': 'tool_call'}]


In [ ]:
from langchain.messages import ToolMessage
# 서울과 뉴욕 각각의 위도,경도를 구하기 위해 tool_calls 는 요소가 2개(서울 좌표, 뉴욕 좌표)

for tool_call in response.tool_calls:
     # 중요: tool_call 구조 분해
    tool_name = tool_call.get("name")       # tool_call["name"]
    tool_args = tool_call.get("args", {})
    tool_call_id = tool_call.get("id")      # ← 이 ID가 매칭 키

    # tool_dict를 사용하여 도구 함수객체 리턴
    selected_tool = tool_dict[tool_name]            # tool_dict를 사용하여 도구 함수객체 리턴
    tool_result = selected_tool.invoke(tool_args)   # globals() 대신에 랭체인에서 함수 실행하기
    print(f'log args ➡ {tool_call["args"]}')       # 도구 호출 시 전달된 인자 출력
    tool_result = selected_tool.invoke(tool_call)
    tool_message = ToolMessage(
                tool_call_id=tool_call_id,      # ← 도구 호출 ID와 매칭
                name=tool_name,                 # ← 도구 이름
                content=str(tool_result)        # ← 도구 실행 결과 (문자열)
    )
    messages.append(tool_message)
messages

서울의 위도는 37.5666791, 경도는 126.9782914입니다.
log args ➡ {'location': '서울'}
서울의 위도는 37.5666791, 경도는 126.9782914입니다.


[SystemMessage(content='너는 날씨를 알려주는 기상캐스터 봇이야.도구에서 받은 날씨 정보를 사람에게 말하듯 자연스럽게 요약해서 알려줘.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='서울 날씨 알려줘', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 155, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEp0paZ3Q1EQjeiiAGh6C41oEqd9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--fb1526af-3f58-4c79-88ee-b4c2fe5f4f55-0', tool_calls=[{'name': 'get_coordinates', 'args': {'location': '서울'}, 'id': 'call_8jWOpy7xZxpOeOwTovNez7PE', 'type': 'tool_call'}], us

In [192]:
response=llm_with_tools.invoke(messages)  # 두번째 함수의 호출 Toolmessage 를 추가하여 요청
print(response.tool_calls)
messages.append(response)

[{'name': 'get_weather_info', 'args': {'lat_lon': '37.5666791,126.9782914'}, 'id': 'call_6lu7Axxb5WUh0YtxaRLeYaxH', 'type': 'tool_call'}]


In [ ]:
from langchain.messages import ToolMessage
# 서울과 뉴욕 각각의 위도,경도를 구하기 위해 tool_calls 는 요소가 2개(서울 좌표, 뉴욕 좌표)

for tool_call in response.tool_calls:
     # 중요: tool_call 구조 분해
    tool_name = tool_call.get("name")       # tool_call["name"]
    tool_args = tool_call.get("args", {})
    tool_call_id = tool_call.get("id")      # ← 이 ID가 매칭 키

    # tool_dict를 사용하여 도구 함수객체 리턴
    selected_tool = tool_dict[tool_name]            # tool_dict를 사용하여 도구 함수객체 리턴
    tool_result = selected_tool.invoke(tool_args)   # globals() 대신에 랭체인에서 함수 실행하기
    print(f'log args ➡ {tool_call["args"]}')       # 도구 호출 시 전달된 인자 출력
    tool_result = selected_tool.invoke(tool_call)
    tool_message = ToolMessage(
                tool_call_id=tool_call_id,      # ← 도구 호출 ID와 매칭
                name=tool_name,                 # ← 도구 이름
                content=str(tool_result)        # ← 도구 실행 결과 (문자열)
    )
    messages.append(tool_message)
messages

37.5666791 126.9782914
log args ➡ {'lat_lon': '37.5666791,126.9782914'}
37.5666791 126.9782914


[SystemMessage(content='너는 날씨를 알려주는 기상캐스터 봇이야.도구에서 받은 날씨 정보를 사람에게 말하듯 자연스럽게 요약해서 알려줘.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='서울 날씨 알려줘', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 155, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEp0paZ3Q1EQjeiiAGh6C41oEqd9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--fb1526af-3f58-4c79-88ee-b4c2fe5f4f55-0', tool_calls=[{'name': 'get_coordinates', 'args': {'location': '서울'}, 'id': 'call_8jWOpy7xZxpOeOwTovNez7PE', 'type': 'tool_call'}], us

In [194]:
response=llm_with_tools.invoke(messages)  
print(response)

content='서울의 현재 날씨는 맑고, 기온은 약 10.8도이며 체감온도는 약 8.5도입니다. 습도는 24%로 건조한 편이에요. 외출할 때 참고하세요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 333, 'total_tokens': 389, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEpAaDd3LmVOubZjNSVI2SRWiOeu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--c8c8b4a9-3bbc-4dd6-b7d8-9e236375f251-0' usage_metadata={'input_tokens': 333, 'output_tokens': 56, 'total_tokens': 389, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [195]:
print(response.content)

서울의 현재 날씨는 맑고, 기온은 약 10.8도이며 체감온도는 약 8.5도입니다. 습도는 24%로 건조한 편이에요. 외출할 때 참고하세요!
